In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import burr12
from scipy.optimize import minimize
import os
np.random.seed(42)


def load_and_clean_data(filepath: str):
    xls = pd.ExcelFile(filepath)
    product_df = xls.parse("Demand")
    dist_df = xls.parse("Demand_Variance")
    product_df.index = product_df.iloc[:, 0]
    product_df = product_df.drop(columns=product_df.columns[0])
    product_df = product_df.T.reset_index(drop=True)
    product_df['ProductID'] = [f'P{i}' for i in range(len(product_df))]
    product_df['Demand'] = pd.to_numeric(product_df['Demand'], errors='coerce')
    product_df['Variance group'] = pd.to_numeric(product_df['Variance group'], downcast='integer', errors='coerce')
    product_df['Margin'] = pd.to_numeric(product_df['Margin'], errors='coerce')
    product_df['COGS'] = pd.to_numeric(product_df['COGS'], errors='coerce')
    product_df['Substitutability group'] = pd.to_numeric(product_df['Substitutability group'], downcast='integer', errors='coerce')
    product_df['Capacity'] = product_df['Capacity'].astype(str).str.rstrip('%')
    product_df['Capacity'] = pd.to_numeric(product_df['Capacity'], errors='coerce')
    product_df['Capacity'] = product_df['Capacity'].apply(lambda x: x if pd.notna(x) else 1.0)
    dist_df.columns = ['DemandVarGroup', 'distribution', 'c', 'd', 'loc', 'scale']
    product_df = product_df.rename(columns={'Variance group': 'DemandVarGroup'})
    product_df = product_df.merge(dist_df.drop(columns='distribution'), on='DemandVarGroup', how='left')
    return product_df

def simulate_demand(product_df: pd.DataFrame, n_samples: int = 500) -> np.ndarray:
    simulated_demand = []
    for _, row in product_df.iterrows():
        dist = burr12(row['c'], row['d'], loc=row['loc'], scale=row['scale'])
        simulated = row['Demand'] * dist.rvs(size=n_samples)
        simulated_demand.append(simulated)
    return np.array(simulated_demand)

def define_objective(demand, margin, cogs, simulated_demand, max_surplus):
    def objective(surplus):
        surplus = np.clip(surplus, 0, max_surplus)
        sales = np.minimum(demand[:, None] + surplus[:, None], simulated_demand)
        revenue = sales * margin[:, None]
        cost = surplus[:, None] * cogs[:, None]
        total_profit = revenue.sum(axis=0) - cost.sum(axis=0)

        if np.any(np.isnan(total_profit)) or np.any(np.isinf(total_profit)):
            return 1e10
        return -np.mean(total_profit)
    return objective

def run_optimization_with_group_constraints(product_df: pd.DataFrame, simulated_demand: np.ndarray, macro_cap: float):
    demand = product_df['Demand'].values
    margin = product_df['Margin'].values
    cogs = product_df['COGS'].values
    capacity = product_df['Capacity'].values
    max_surplus = capacity * demand
    total_demand = demand.sum()
    n_products = len(demand)

    objective_fn = define_objective(demand, margin, cogs, simulated_demand, max_surplus)

    expected_actual = simulated_demand.mean(axis=1)
    overflow = np.maximum(expected_actual - demand, 0)
    sub_groups = product_df.groupby('Substitutability group').groups

    group_constraints = []
    for group_id, indices in sub_groups.items():
        valid_idx = [i for i in indices if max_surplus[i] > 0]
        if len(valid_idx) > 1:
            idx = list(valid_idx)
            required = overflow[idx].sum() * ss
            group_constraints.append({
                'type': 'ineq',
                'fun': lambda x, idx=idx, req=required: np.sum(x[idx]) - req
            })

    constraints = [{'type': 'ineq', 'fun': lambda x: macro_cap * total_demand - np.sum(x)}]  + group_constraints
    bounds = [(0, ms if np.isfinite(ms) else None) for ms in max_surplus]
    x0 = 0.2 * max_surplus
    # x0 = np.zeros(n_products)
    x0[max_surplus == 0] = 0


    result = minimize(
        objective_fn,
        x0,
        method=method,
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000, 'disp': True}
    )

    return result, max_surplus, overflow, sub_groups

def save_results_and_visualize(product_df, result, max_surplus, simulated_demand, overflow, sub_groups, output_dir="SLSQP_SoftConstraints"):
    os.makedirs(output_dir, exist_ok=True)

    demand = product_df['Demand'].values
    margin = product_df['Margin'].values
    cogs = product_df['COGS'].values

    product_df['OptimalSurplus'] = result.x
    product_df['MaxSurplus'] = max_surplus

    output_csv = os.path.join(output_dir, "optimal_surplus_output.csv")
    product_df[['ProductID', 'Demand', 'Margin', 'COGS', 'Capacity', 'OptimalSurplus']].to_csv(output_csv, index=False)

    sales = np.minimum(demand[:, None] + result.x[:, None], simulated_demand)
    revenue = sales * margin[:, None]
    cost = result.x[:, None] * cogs[:, None]
    profit = revenue.sum(axis=0) - cost.sum(axis=0)

    pd.DataFrame({'Profit': profit}).to_csv(os.path.join(output_dir, "profit.csv"), index=False)

    group_data = []
    for group_id, indices in sub_groups.items():
        idx = list(indices)
        group_surplus = result.x[idx].sum()
        group_need = overflow[idx].sum()
        gap = group_need - group_surplus
        group_data.append({'Group': group_id, 'Surplus': group_surplus, 'Need': group_need, 'Gap': gap})
        print(f"Group {group_id}: Surplus = {group_surplus:.2f}, Needed = {group_need:.2f}, Gap = {gap:.2f}")

    pd.DataFrame(group_data).to_csv(os.path.join(output_dir, "group_summary.csv"), index=False)

    print(f"\nOptimization finished: {'Success' if result.success else 'Not converged'}")
    print("Message:", result.message)

    # === Plots ===
    plt.figure(figsize=(14, 6))
    plt.bar(product_df['ProductID'], result.x, color='royalblue')
    # plt.xticks(rotation=90)
    plt.xticks(ticks=np.arange(0, len(product_df), 10), labels=product_df['ProductID'][::10], rotation=45)
    plt.ylabel('Surplus Units')
    plt.title('Optimal Surplus per Product')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "surplus_per_product.png"))
    plt.close()

    plt.hist(profit, bins=30, color='green', edgecolor='black')
    plt.title('Histogram of Simulated Total Profits')
    plt.xlabel('Total Profit')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "profit_distribution.png"))
    plt.close()

    print("\nMean Profit:", np.mean(profit))
    print("Std Dev Profit:", np.std(profit))
    print("Saved all outputs to:", output_dir)


def main():
    filepath = r"C:\Users\hadad\OneDrive - Iowa State University\personal\Corteva\Input_SUA.xlsx"
    product_df = load_and_clean_data(filepath)
    # product_df = product_df.iloc[:100]
    simulated_demand = simulate_demand(product_df, n_samples)
    result, max_surplus, overflow, sub_groups = run_optimization_with_group_constraints(product_df, simulated_demand, macro_cap)
    save_results_and_visualize(product_df, result, max_surplus, simulated_demand, overflow, sub_groups)

if __name__ == "__main__":
    method = 'SLSQP'
    macro_cap = 0.4
    n_samples = 100
    ss = 0.01
    print("method:", method)
    print("macro_cap:", macro_cap)
    print("n_samples:", n_samples)
    print("ss:", ss)
    main()
    
# Mean Expected Profit: $3.63 Billion

# Standard Deviation: $67 Million
# (From 100 Monte Carlo simulations)

method: SLSQP
macro_cap: 0.4
n_samples: 100
ss: 0.01


c:\Users\hadad\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -3634963656.6205516
            Iterations: 521
            Function evaluations: 237272
            Gradient evaluations: 519
Group 0: Surplus = 32762.01, Needed = 30340.49, Gap = -2421.52
Group 1: Surplus = 13880.47, Needed = 6711.67, Gap = -7168.80
Group 2: Surplus = 34380.88, Needed = 53355.56, Gap = 18974.68
Group 3: Surplus = 36436.32, Needed = 6863.42, Gap = -29572.90
Group 4: Surplus = 10481.59, Needed = 2279.92, Gap = -8201.67
Group 5: Surplus = 1940.41, Needed = 0.00, Gap = -1940.41
Group 6: Surplus = 13232.69, Needed = 22007.98, Gap = 8775.29
Group 7: Surplus = 15813.34, Needed = 5641.90, Gap = -10171.44
Group 8: Surplus = 22064.16, Needed = 2231.39, Gap = -19832.77
Group 9: Surplus = 75748.75, Needed = 72482.62, Gap = -3266.13
Group 10: Surplus = 3824.98, Needed = 4298.70, Gap = 473.72
Group 11: Surplus = 11910.59, Needed = 15495.41, Gap = 3584.82
Group 12: Surplus = 9453.74, Needed = 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import burr12
from scipy.optimize import minimize
import os
np.random.seed(42)


def load_and_clean_data(filepath: str):
    xls = pd.ExcelFile(filepath)
    product_df = xls.parse("Demand")
    dist_df = xls.parse("Demand_Variance")
    product_df.index = product_df.iloc[:, 0]
    product_df = product_df.drop(columns=product_df.columns[0])
    product_df = product_df.T.reset_index(drop=True)
    product_df['ProductID'] = [f'P{i}' for i in range(len(product_df))]
    product_df['Demand'] = pd.to_numeric(product_df['Demand'], errors='coerce')
    product_df['Variance group'] = pd.to_numeric(product_df['Variance group'], downcast='integer', errors='coerce')
    product_df['Margin'] = pd.to_numeric(product_df['Margin'], errors='coerce')
    product_df['COGS'] = pd.to_numeric(product_df['COGS'], errors='coerce')
    product_df['Substitutability group'] = pd.to_numeric(product_df['Substitutability group'], downcast='integer', errors='coerce')
    product_df['Capacity'] = product_df['Capacity'].astype(str).str.rstrip('%')
    product_df['Capacity'] = pd.to_numeric(product_df['Capacity'], errors='coerce')
    product_df['Capacity'] = product_df['Capacity'].apply(lambda x: x if pd.notna(x) else 1.0)
    dist_df.columns = ['DemandVarGroup', 'distribution', 'c', 'd', 'loc', 'scale']
    product_df = product_df.rename(columns={'Variance group': 'DemandVarGroup'})
    product_df = product_df.merge(dist_df.drop(columns='distribution'), on='DemandVarGroup', how='left')
    return product_df

def simulate_demand(product_df: pd.DataFrame, n_samples: int = 500) -> np.ndarray:
    simulated_demand = []
    for _, row in product_df.iterrows():
        dist = burr12(row['c'], row['d'], loc=row['loc'], scale=row['scale'])
        simulated = row['Demand'] * dist.rvs(size=n_samples)
        simulated_demand.append(simulated)
    return np.array(simulated_demand)

def define_objective(demand, margin, cogs, simulated_demand, max_surplus):
    def objective(surplus):
        surplus = np.clip(surplus, 0, max_surplus)
        sales = np.minimum(demand[:, None] + surplus[:, None], simulated_demand)
        revenue = sales * margin[:, None]
        cost = surplus[:, None] * cogs[:, None]
        total_profit = revenue.sum(axis=0) - cost.sum(axis=0)

        if np.any(np.isnan(total_profit)) or np.any(np.isinf(total_profit)):
            return 1e10
        return -np.mean(total_profit)
    return objective

def run_optimization_with_group_constraints(product_df: pd.DataFrame, simulated_demand: np.ndarray, macro_cap: float):
    demand = product_df['Demand'].values
    margin = product_df['Margin'].values
    cogs = product_df['COGS'].values
    capacity = product_df['Capacity'].values
    max_surplus = capacity * demand
    total_demand = demand.sum()
    n_products = len(demand)

    objective_fn = define_objective(demand, margin, cogs, simulated_demand, max_surplus)

    expected_actual = simulated_demand.mean(axis=1)
    overflow = np.maximum(expected_actual - demand, 0)
    sub_groups = product_df.groupby('Substitutability group').groups

    group_constraints = []
    for group_id, indices in sub_groups.items():
        valid_idx = [i for i in indices if max_surplus[i] > 0]
        if len(valid_idx) > 1:
            idx = list(valid_idx)
            required = overflow[idx].sum() * ss
            group_constraints.append({
                'type': 'ineq',
                'fun': lambda x, idx=idx, req=required: np.sum(x[idx]) - req
            })

    constraints = [{'type': 'ineq', 'fun': lambda x: macro_cap * total_demand - np.sum(x)}]  + group_constraints
    bounds = [(0, ms if np.isfinite(ms) else None) for ms in max_surplus]
    x0 = 0.2 * max_surplus
    # x0 = np.zeros(n_products)
    x0[max_surplus == 0] = 0


    result = minimize(
        objective_fn,
        x0,
        method=method,
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000, 'disp': True}
    )

    return result, max_surplus, overflow, sub_groups

def save_results_and_visualize(product_df, result, max_surplus, simulated_demand, overflow, sub_groups, output_dir=f"SLSQP_SoftConstraints_gc{ss}"):
    os.makedirs(output_dir, exist_ok=True)

    demand = product_df['Demand'].values
    margin = product_df['Margin'].values
    cogs = product_df['COGS'].values

    product_df['OptimalSurplus'] = result.x
    product_df['MaxSurplus'] = max_surplus

    output_csv = os.path.join(output_dir, "optimal_surplus_output.csv")
    product_df[['ProductID', 'Demand', 'Margin', 'COGS', 'Capacity', 'OptimalSurplus']].to_csv(output_csv, index=False)

    sales = np.minimum(demand[:, None] + result.x[:, None], simulated_demand)
    revenue = sales * margin[:, None]
    cost = result.x[:, None] * cogs[:, None]
    profit = revenue.sum(axis=0) - cost.sum(axis=0)

    pd.DataFrame({'Profit': profit}).to_csv(os.path.join(output_dir, "profit.csv"), index=False)

    group_data = []
    for group_id, indices in sub_groups.items():
        idx = list(indices)
        group_surplus = result.x[idx].sum()
        group_need = overflow[idx].sum()
        gap = group_need - group_surplus
        group_data.append({'Group': group_id, 'Surplus': group_surplus, 'Need': group_need, 'Gap': gap})
        print(f"Group {group_id}: Surplus = {group_surplus:.2f}, Needed = {group_need:.2f}, Gap = {gap:.2f}")

    pd.DataFrame(group_data).to_csv(os.path.join(output_dir, "group_summary.csv"), index=False)

    print(f"\nOptimization finished: {'Success' if result.success else 'Not converged'}")
    print("Message:", result.message)

    # === Plots ===
    plt.figure(figsize=(14, 6))
    plt.bar(product_df['ProductID'], result.x, color='royalblue')
    # plt.xticks(rotation=90)
    plt.xticks(ticks=np.arange(0, len(product_df), 10), labels=product_df['ProductID'][::10], rotation=45)
    plt.ylabel('Surplus Units')
    plt.title('Optimal Surplus per Product')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "surplus_per_product.png"))
    plt.close()

    plt.hist(profit, bins=30, color='green', edgecolor='black')
    plt.title('Histogram of Simulated Total Profits')
    plt.xlabel('Total Profit')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "profit_distribution.png"))
    plt.close()

    print("\nMean Profit:", np.mean(profit))
    print("Std Dev Profit:", np.std(profit))
    print("Saved all outputs to:", output_dir)


def main():
    filepath = r"C:\Users\hadad\OneDrive - Iowa State University\personal\Corteva\Input_SUA.xlsx"
    product_df = load_and_clean_data(filepath)
    # product_df = product_df.iloc[:100]
    simulated_demand = simulate_demand(product_df, n_samples)
    result, max_surplus, overflow, sub_groups = run_optimization_with_group_constraints(product_df, simulated_demand, macro_cap)
    save_results_and_visualize(product_df, result, max_surplus, simulated_demand, overflow, sub_groups)

if __name__ == "__main__":
    method = 'SLSQP'
    macro_cap = 0.4
    n_samples = 100
    ss = 0.1
    print("method:", method)
    print("macro_cap:", macro_cap)
    print("n_samples:", n_samples)
    print("ss:", ss)
    main()

# Mean Expected Profit: $3.64 Billion

# Standard Deviation: $69 Million
# (From 100 Monte Carlo simulations)
    

method: SLSQP
macro_cap: 0.4
n_samples: 100
ss: 0.1
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3640054461.219176
            Iterations: 636
            Function evaluations: 288899
            Gradient evaluations: 632
Group 0: Surplus = 40860.19, Needed = 30340.49, Gap = -10519.70
Group 1: Surplus = 14193.65, Needed = 6711.67, Gap = -7481.98
Group 2: Surplus = 37687.16, Needed = 53355.56, Gap = 15668.40
Group 3: Surplus = 41706.66, Needed = 6863.42, Gap = -34843.24
Group 4: Surplus = 10492.92, Needed = 2279.92, Gap = -8213.00
Group 5: Surplus = 3074.42, Needed = 0.00, Gap = -3074.42
Group 6: Surplus = 13295.65, Needed = 22007.98, Gap = 8712.33
Group 7: Surplus = 16497.44, Needed = 5641.90, Gap = -10855.55
Group 8: Surplus = 32597.61, Needed = 2231.39, Gap = -30366.21
Group 9: Surplus = 78423.77, Needed = 72482.62, Gap = -5941.16
Group 10: Surplus = 7075.08, Needed = 4298.70, Gap = -2776.38
Group 11: Surplus = 11910.60, Needed = 15495.41